<a href="https://colab.research.google.com/github/netjerikhet/ai-human-emotions/blob/main/Photo_Animation_v0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ffmpy &> /dev/null
!git init -q .
!git remote add origin https://github.com/AliaksandrSiarohin/first-order-model
!git pull -q origin master
!git clone -q https://github.com/graphemecluster/first-order-model-demo demo 

In [5]:
!pip install imageio matplotlib numpy pandas python-dateutil pytz PyYAML scikit-image scikit-learn scipy torch torchvision tqdm

In [28]:
!pip install face-alignment #face-alignment

  Created wheel for face-alignment: filename=face_alignment-1.3.5-py2.py3-none-any.whl size=28241 sha256=1104a6d626672ac89f59ac08e3dfb08fc2acb420ec1a99142a07bfdf773cc27c
  Stored in directory: /root/.cache/pip/wheels/c9/ba/4d/2d368f55e5f929f9472da59e356fbdf1483f885de80a5bc620
Successfully built face-alignment


In [24]:
!pip install deepface #deepface

In [3]:
# !sudo apt update

In [4]:
# !sudo apt install apt-transport-https ca-certificates curl software-properties-common
# !curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo apt-key add -
# !sudo add-apt-repository "deb [arch=amd64] https://download.docker.com/linux/ubuntu bionic stable"
# !sudo apt update
# !sudo apt install docker-ce

In [5]:
# !docker build -t first-order-model .

In [3]:
!gdown --id 1_v_xW1V52gZCZnXgh1Ap_gwA9YVIzUnS

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1_v_xW1V52gZCZnXgh1Ap_gwA9YVIzUnS
To: /content/vox-cpk.pth.tar
100% 729M/729M [00:08<00:00, 83.4MB/s]


In [6]:
# !gzip vox-cpk.pth.tar
!mkdir -p data/vox-png

In [7]:
!CUDA_VISIBLE_DEVICES=0 python run.py --config config/vox-256.yaml --mode animate --checkpoint vox-cpk.pth.tar

/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Use random train-test split.
Traceback (most recent call last):
  File "run.py", line 72, in <module>
    dataset = FramesDataset(is_train=(opt.mode == 'train'), **config['dataset_params'])
  File "/content/frames_dataset.py", line 83, in __init__
    train_videos, test_videos = train_test_split(self.videos, random_state=random_seed, test_size=0.2)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py", line 2421, in train_test_split
    n_samples, test_size, train_size, default_test_size=0.25
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py", line 2101, in _validate_shuffle_split
    "aforementioned parameters.".format(n

In [8]:
import os, requests, numpy
import imageio
import IPython.display
import PIL.Image
import cv2
import imageio
import io
import ipywidgets
import os.path
import requests
import skimage.transform
import warnings
from demo import load_checkpoints, make_animation
from ffmpy import FFmpeg
from skimage import img_as_ubyte
import skimage.transform

In [9]:
!git clone -q https://github.com/ChanduS06/dataset-generic Emotions-dir

In [16]:
def resize(image, size=(256, 256)):
    w, h = image.size
    d = min(w, h)
    r = ((w - d) // 2, (h - d) // 2, (w + d) // 2, (h + d) // 2)
    return image.resize(size, resample=PIL.Image.LANCZOS, box=r)

In [18]:
from google.colab import files as FILE
import os

img_data = requests.get("https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSa1INgadp1f7fPVyip1SJ4Z4nJRO6L6ZarPRK1UeZEhPHnegTEK8-v6st82lDk1XO5D3M&usqp=CAU").content
with open('einstein.png', 'wb') as handler:
    handler.write(img_data)

img_data_m_obama = requests.get("https://aisrtl-a.akamaihd.net/themenarchiv/michelle-obama-bilder/3-1e1eaqaq/300x0/2781/michelle-obama-t4139.jpg").content
with open('m_obama.jpg', 'wb') as handler:
    handler.write(img_data_m_obama)

selected_image = resize(PIL.Image.open('einstein.png').convert("RGB"))

In [14]:
selected_video = "Emotions-dir/Emotions-dir/Angry/Female/W025_light_uniform_angry_camera_front.mp4"

In [26]:
# For Deepface verification method only
from deepface import DeepFace

metrics = ["cosine", "euclidean", "euclidean_l2"]

Directory  /root /.deepface created
Directory  /root /.deepface/weights created


In [29]:
# for Face-alignment verification only
import face_alignment

# sfd for SFD, dlib for Dlib and folder for existing bounding boxes.
fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, face_detector='sfd')

Downloading: "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" to /root/.cache/torch/hub/checkpoints/s3fd-619a316812.pth


  0%|          | 0.00/85.7M [00:00<?, ?B/s]

Downloading: "https://www.adrianbulat.com/downloads/python-fan/2DFAN4-cd938726ad.zip" to /root/.cache/torch/hub/checkpoints/2DFAN4-cd938726ad.zip


  0%|          | 0.00/91.9M [00:00<?, ?B/s]

In [31]:
os.makedirs("user", exist_ok=True)
def perform_animation():
	filename = 'vox-cpk.pth.tar' #model.value + ('' if model.value == 'fashion' else '-cpk') + '.pth.tar'
	if not os.path.isfile(filename):
		download = requests.get(requests.get('https://cloud-api.yandex.net/v1/disk/public/resources/download?public_key=https://yadi.sk/d/lEw8uRm140L_eQ&path=/' + filename).json().get('href'))
		with open(filename, 'wb') as checkpoint:
			checkpoint.write(download.content)
	reader = imageio.get_reader(selected_video, mode='I', format='FFMPEG')
	fps = reader.get_meta_data()['fps']
	driving_video = []
	for frame in reader:
		driving_video.append(frame)
	generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', checkpoint_path=filename)
	predictions = make_animation(
		skimage.transform.resize(numpy.asarray(selected_image), (256, 256)),
		[skimage.transform.resize(frame, (256, 256)) for frame in driving_video],
		generator,
		kp_detector,
		relative=True,
		adapt_movement_scale=False
	)
	if selected_video.startswith('user/') or selected_video == 'demo/videos/0.mp4':
		imageio.mimsave('temp.mp4', [img_as_ubyte(frame) for frame in predictions], fps=fps)
		FFmpeg(inputs={'temp.mp4': None, selected_video: None}, outputs={'output.mp4': '-c copy -y'}).run()
	else:
		imageio.mimsave('output.mp4', [img_as_ubyte(frame) for frame in predictions], fps=fps)

	# distance = []

	# #verification and validation using deepface
	# for i in predictions:
	# 	result = DeepFace.verify(img1_path = selected_image, img2_path = img_as_ubyte(i), distance_metric = metrics[1])
	# 	distance.append(result)
	
	# print(distance)

perform_animation()

100%|██████████| 240/240 [00:45<00:00,  5.23it/s]
